# Basic Interface Tutorial

# An introduction to GETTSIM

## Part 1: What is GETTSIM and how does it work?

You may find an installation guide [here](gettsim.readthedocs.io/en/latest/tutorials/installation.html)

GETTSIM provides an almost complete simulation of the german tax and transfer system. In this very introductory part of the tutorial we will focus on two central functions in GETTSIM: `get_policies_for_date` and `compute_taxes_and_transfers`, so in a first step, we import both:

In [ ]:
import pandas as pd

from gettsim import get_policies_for_date
from gettsim import compute_taxes_and_transfers

GETTSIM might be interesting for both students and scientists. While students might use GETTSIM as an introduction to the german tax and transfer system or as a part of a project or a thesis, scientists can use it for advanced microsimulations.

This first part of the tutorial is designed to be a more general and very basic introduction for newcomers. After having a look at GETTSIM with very simple data structures here, the tutorial proceeds in part 2 by applying GETTSIM on other types of data and creating a simulated set of data, while part 3 and 4 concentartes on adjusting GETTSIM's parameters and function, to e.g. implement a policy reform.

In GETTSIM, there are 41 input variables, as especially the transfers depend on a lot of different variables. There is a detailed (**but somewhat wrong?**) list of them [here](https://gettsim.readthedocs.io/en/latest/crosswalk.html). But for now, as an example, we will have a look on those needed to calculate the income tax and the social security contributions for an individual. For this purpose, we define a data frame with them:

#### Create data for one individual

In [ ]:
data = pd.DataFrame(
    {
        "p_id": 1,
        "tu_id": 1,
        "hh_id": 1,
        "bruttolohn_m": 2000,
        "_zu_verst_eink_kein_kinderfreib_tu": 24000,
        "_zu_verst_kapital_eink_tu": 0,
        "wohnort_ost": False,
        "kind": False,
        "alter": 35,
        "selbstständig": False,
        "hat_kinder": False,
        "eink_selbst_m": 0,
        "ges_rente_m": 0,
        "prv_krankenv": False,
        "kinderfreib_tu": 0,
        "_kindergeld_m_tu_basis": 0,
    },
    index=[0],
)
# Transpose data for better readability
data.T

As you can see, there are three identifiers: `p_id` identifies a person, `tu_id` a transfer unit and `hh_id` a household.

`_m` means, that the variable labeled with this suffix is a monthly variable. Variables without this suffix are always on a yearly basis.

In the same way, `_tu` and `_hh` mean, that the variable is to be interpreted on transfer unit level, or on houshold level, respectively. Variables without these suffixes are always on individual level.

#### Get parameters and functions

With GETTSIM it is possible to compute taxes and transfers back to 2010. As the functions and parameters used by GETTSIM vary through the years, it is necessary to specify the year we are interested in. To do so, we have to use the function `get_policies_for_date` and take our year as a string:

In [ ]:
params_dict, policy_func_dict = get_policies_for_date("2020")

The function returns two values: ```policy_func_dict``` contains the relevant functions for the year 2020. ```params_dict``` contains the relevant parameters. 

As for the moment we only have a look at a part of GETTSIM, we define and will pass a list with some with variables which GETTSIM would normally calculate based on a load of further variables, but which we want GETTSIM to take as given. You can see the values of them in the data frame we defined.

In [ ]:
columns = [
    "_kindergeld_m_tu_basis",
    "kinderfreib_tu",
    "_zu_verst_kapital_eink_tu",
    "_zu_verst_eink_kein_kinderfreib_tu",
]

**Defining targets**

Now we have to specify the variables we want GETTSIM to return, as without specifying them GETTSIM would try to compute every single variable in the tax and transfer system, which is not possible with our input data. For this purpose we define another list with the variables we are interested in (a list with all variables GETTSIM is able to compute can be found [here](https://gettsim.readthedocs.io/en/latest/crosswalk.html)).

In [ ]:
targets = [
    "eink_st_tu",
    "soli_st_tu",
    "ges_krankenv_beitr_m",
    "rentenv_beitr_m",
    "arbeitsl_v_beitr_m",
    "pflegev_beitr_m",
]

As you may see from the variable names, we focus on the income tax, the Solidaritätszuschlag and the four social security contributions. Now we are approaching the core of it all: The function `compute_taxes_and_transfers` will now compute those targets, based on the input for the individual we defined.

#### Compute targets

In [ ]:
result = compute_taxes_and_transfers(
    data=data,
    user_functions=policy_func_dict,
    user_columns=columns,
    params=params_dict,
    targets=targets,
)
result.round(2).T

As you can see, the function `compute_taxes_and_transfers` takes five inputs: Our data frame (`data`), the relevant functions (`policy_func_dict`) which, as you remember, are one of the two returns of the function `get_policies_for_date`, our additional pre-defined variables (`columns`), the other return of `get_policies_for_date` which are the relevant parameters (`params_dict`) and of course the targets we want to compute (`targets`). `result` is the data frame that contains the results for our individual.

It is of course possible to join the data and the results:

In [ ]:
data1=data.join(result)
data1

If you are interested in which *parameters* determine a certain transfer, e.g. Kindergeld, you can access them the following way:

In [ ]:
print(*params_dict.keys(),sep="\n")

In [ ]:
print(*params_dict["kindergeld"].keys(), sep="\n")

In [ ]:
params_dict["kindergeld"]["kindergeld"]


These parameters determine the Kindergeld.

In Part 2 of the tutorial we will introduce other data structures to the use of GETTSIM and have a look on which data structure GETTSIM requires.